### Data visualisation

Define a visual representation for the experimental measurements of the kernel perceptron implementation, then convert them to Tikz using the `tikzplotlib` library. The only graphs needed are heatmaps and line graphs. Save the resulting .tex (and .png for the heatmaps) in `OUTPUT`.

In [232]:
import sys
sys.path.append("../")

In [233]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tikzplotlib
import json

import os
import shutil

from KMeans import REDUCTIONS
from experiments import RESULTS_LOCATION
from utils import APPROACHES

TIKZ_GRAPHS = "graphs"
results = json.load(open(f".{RESULTS_LOCATION}"))

In [234]:
# tikzplotlib in some plots has issue rendering the labels,
# they need to be written in LateX notation

def fix(labels):
    return [f"${label}$" for label in labels]

def posneg(l):
    return [0] + sum([[v, -v] for v in range(1, l)], start=[])

Find maximum between training and test error

In [235]:
max_test_error = max(max(app[approach]["test_error"] for approach in APPROACHES) for app in results.values())
max_training_error = max(max(app[approach]["training_error_km"] for approach in APPROACHES) for app in results.values())

max_error = max(max_test_error, max_training_error)

Convert matplotlib plots into tikz plots

In [236]:
if os.path.exists(TIKZ_GRAPHS):
    shutil.rmtree(TIKZ_GRAPHS)

os.mkdir(TIKZ_GRAPHS)

colors  = ["#ffb2b2", "#ecd9c6", "#b2b2ff"]
borders = ["#ff0000", "#734c26", "#0000ff"]

In [237]:
def double_bar(attribute, label, filename):
	plt.rcParams['figure.figsize'] = [10, 8]
	width = 0.6

	errors = [[k[approach][attribute] for k in results.values()] for approach in APPROACHES]

	x = np.arange(len(REDUCTIONS))
	y = np.linspace(0, max_error, 7)

	for index, approach, error in zip(posneg(len(APPROACHES)), APPROACHES, errors):
		plt.bar(x + index * (width / len(REDUCTIONS)), error, width / len(REDUCTIONS), label=approach, color=colors[index], edgecolor=borders[index])

	plt.xticks(x, fix(REDUCTIONS))
	plt.yticks(y, fix([f"{f:.2f}" for f in y]))
	plt.xlabel("$k$")
	plt.ylabel(label)
	plt.grid(which="major", axis="y")
	plt.legend()

	# plt.show()

	tikzplotlib.save(f"{TIKZ_GRAPHS}/{filename}")
	plt.close()


In [238]:
double_bar("test_error", "Test error", "test-error.tex")
double_bar("training_error_km", "Training error", "training-error.tex")